#**Ingesta a través de API rest**

#**Configuración de AWS CLI en Colab**


In [ ]:
##Instalación de boto3 & request para la API de los datos
!pip install boto3
!pip install requests

In [ ]:
import requests
import boto3
import json
from io import StringIO
import pandas as pd

In [ ]:
##Parametros de conexión con AWS###
!pip install awscli
!pip install --upgrade awscli

aws_access_key_id='____________________'
aws_secret_access_key='________________________________________'
aws_session_token='_____________________________________________________________'

!aws configure list --profile assumed-role

      Name                    Value             Type    Location
      ----                    -----             ----    --------
   profile             assumed-role           manual    --profile

The config profile (assumed-role) could not be found


In [ ]:
##Parametros del bucket##
s3_bucket='rawzonetrabajo1'
s3_target_file_name='raw/annual_surface_temp_change.csv'

#**Obteniendo la información del API que tiene el IMF**

####**Annual Surface Temperature Change**

In [ ]:
url_temp_change = "https://services9.arcgis.com/weJ1QsnbMYJlCHdG/arcgis/rest/services/Indicator_3_1_Climate_Indicators_Annual_Mean_Global_Surface_Temperature/FeatureServer/0/query?where=1%3D1&outFields=*&outSR=4326&f=json"

response_temp_change = requests.get(url_temp_change)
data_temp_change= response_temp_change.json()

features_temp_change = data_temp_change['features']

df_temp_change= pd.json_normalize(features_temp_change, record_path=None, meta=['id', 'geometry'], errors='ignore')

df_temp_change = pd.DataFrame([feature.get('properties', {}) for feature in features_temp_change])